Se le proporcionan datos de comportamiento de clientes de Santander de un año y medio para predecir qué nuevos productos adquirirán. Los datos comienzan el 28/01/2015 e incluyen registros mensuales de los productos que posee un cliente, como tarjetas de crédito, cuentas de ahorro, etc. Predecirá qué productos adicionales adquirirá un cliente durante el último mes, el 28/06/2016, además de los que ya tenía el 28/05/2016. Estos productos corresponden a las columnas ind_(xyz)_ult1, que corresponden a las columnas 25 a 48 en los datos de entrenamiento. Predecirá qué comprará un cliente además de lo que ya tenía el 28/05/2016.


In [1]:
import pandas as pd
import opendatasets as od
od.download('https://www.kaggle.com/competitions/santander-product-recommendation/data')


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading santander-product-recommendation.zip to ./santander-product-recommendation


100%|██████████| 228M/228M [00:00<00:00, 1.02GB/s]



Extracting archive ./santander-product-recommendation/santander-product-recommendation.zip to ./santander-product-recommendation


In [2]:
import zipfile
import os

In [6]:
# Ruta del archivo ZIP descargado
zip_path_two = "/workspaces/proyecto-final/src/santander-product-recommendation/train_ver2.csv.zip"

# Directorio donde se va a extraer
extract_dir_two = "src/"


# Extraer el archivo ZIP
if os.path.exists(zip_path_two):
    with zipfile.ZipFile(zip_path_two, 'r') as zip_ref:
        zip_ref.extractall(extract_dir_two)
        print(f"✅ Archivo extraído en: {extract_dir_two}")
else:
    print(f"No se encontró el archivo ZIP en: {zip_path_two}")

✅ Archivo extraído en: src/


## Determinar archivo por lotes 

In [7]:
def analizar_chunks_csv_plano(ruta_csv, chunksize=100000):
    import pandas as pd
    import os
    import numpy as np

    reader = pd.read_csv(ruta_csv, chunksize=chunksize, low_memory=False)
    primer_chunk = next(reader)

    tamano_ram_MB = primer_chunk.memory_usage(deep=True).sum() / 1024**2
    filas_chunk = len(primer_chunk)

    # Conteo real de filas
    total_filas_estimadas = sum(1 for _ in open(ruta_csv)) - 1  # resta 1 por el header
    total_chunks_estimados = int(np.ceil(total_filas_estimadas / chunksize))
    ram_total_estimada_MB = total_chunks_estimados * tamano_ram_MB

    archivo_MB = os.path.getsize(ruta_csv) / 1024**2

    print(f"🔍 Tamaño del archivo CSV plano: {archivo_MB:.2f} MB")
    print(f"📦 Chunk size: {chunksize} filas")
    print(f"🧠 RAM usada por 1 chunk: {tamano_ram_MB:.2f} MB")
    print(f"📊 Estimación total de filas: {total_filas_estimadas:,}")
    print(f"📈 Número de chunks estimado: {total_chunks_estimados}")
    print(f"💾 Estimación total de uso de RAM si se cargara todo: {ram_total_estimada_MB:.2f} MB")

    return {
        "chunk_rows": filas_chunk,
        "chunk_ram_MB": tamano_ram_MB,
        "total_rows": total_filas_estimadas,
        "estimated_total_chunks": total_chunks_estimados,
        "estimated_total_ram_MB": ram_total_estimada_MB
    }


In [11]:
ruta = '/workspaces/proyecto-final/src/santander-product-recommendation/train_ver2.csv'
resumen = analizar_chunks_csv_plano(ruta)

🔍 Tamaño del archivo CSV plano: 2186.55 MB
📦 Chunk size: 100000 filas
🧠 RAM usada por 1 chunk: 110.55 MB
📊 Estimación total de filas: 13,647,309
📈 Número de chunks estimado: 137
💾 Estimación total de uso de RAM si se cargara todo: 15145.52 MB


## Guardar por lotes del archivo .csv a la base de datos

In [12]:
import pandas as pd
import sqlite3

# Ruta del archivo CSV grande
ruta_csv = '/workspaces/proyecto-final/src/santander-product-recommendation/train_ver2.csv'

# Conexión a la base de datos SQLite
con = sqlite3.connect("Proyecto.db")

# Leer por chunks
chunksize = 100000
i = 0

for chunk in pd.read_csv(ruta_csv, chunksize=chunksize, low_memory=False):
    i += 1
    print(f"🔄 Guardando chunk {i}...")

    # Guardar cada chunk en una tabla llamada 'santander'
    chunk.to_sql("santander", con, if_exists="append", index=False)

print("✅ Todos los chunks han sido cargados en la base de datos.")


🔄 Guardando chunk 1...
🔄 Guardando chunk 2...
🔄 Guardando chunk 3...
🔄 Guardando chunk 4...
🔄 Guardando chunk 5...
🔄 Guardando chunk 6...
🔄 Guardando chunk 7...
🔄 Guardando chunk 8...
🔄 Guardando chunk 9...
🔄 Guardando chunk 10...
🔄 Guardando chunk 11...
🔄 Guardando chunk 12...
🔄 Guardando chunk 13...
🔄 Guardando chunk 14...
🔄 Guardando chunk 15...
🔄 Guardando chunk 16...
🔄 Guardando chunk 17...
🔄 Guardando chunk 18...
🔄 Guardando chunk 19...
🔄 Guardando chunk 20...
🔄 Guardando chunk 21...
🔄 Guardando chunk 22...
🔄 Guardando chunk 23...
🔄 Guardando chunk 24...
🔄 Guardando chunk 25...
🔄 Guardando chunk 26...
🔄 Guardando chunk 27...
🔄 Guardando chunk 28...
🔄 Guardando chunk 29...
🔄 Guardando chunk 30...
🔄 Guardando chunk 31...
🔄 Guardando chunk 32...
🔄 Guardando chunk 33...
🔄 Guardando chunk 34...
🔄 Guardando chunk 35...
🔄 Guardando chunk 36...
🔄 Guardando chunk 37...
🔄 Guardando chunk 38...
🔄 Guardando chunk 39...
🔄 Guardando chunk 40...
🔄 Guardando chunk 41...
🔄 Guardando chunk 42...
🔄

## Revisar y confirmar información en base de datos

In [13]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
con = sqlite3.connect("Proyecto.db")


In [16]:
#  cuántas filas hay
total = pd.read_sql("SELECT COUNT(*) AS total_filas FROM santander;", con)
print(total)


   total_filas
0     13647309


In [15]:
primeros = pd.read_sql("SELECT * FROM santander LIMIT 5;", con)
print(primeros)


   fecha_dato  ncodpers ind_empleado pais_residencia sexo  age  fecha_alta  \
0  2015-01-28   1375586            N              ES    H   35  2015-01-12   
1  2015-01-28   1050611            N              ES    V   23  2012-08-10   
2  2015-01-28   1050612            N              ES    V   23  2012-08-10   
3  2015-01-28   1050613            N              ES    H   22  2012-08-10   
4  2015-01-28   1050614            N              ES    V   23  2012-08-10   

   ind_nuevo antiguedad  indrel  ... ind_hip_fin_ult1  ind_plan_fin_ult1  \
0        0.0          6     1.0  ...                0                  0   
1        0.0         35     1.0  ...                0                  0   
2        0.0         35     1.0  ...                0                  0   
3        0.0         35     1.0  ...                0                  0   
4        0.0         35     1.0  ...                0                  0   

  ind_pres_fin_ult1 ind_reca_fin_ult1 ind_tjcr_fin_ult1 ind_valo_fin_ult1 

## Creación de indices para optimizar selects

In [17]:
import sqlite3

con = sqlite3.connect("Proyecto.db")
cursor = con.cursor()

# Crear índice por cliente
cursor.execute("CREATE INDEX IF NOT EXISTS idx_cliente ON santander(ncodpers);")

# Crear índice por fecha
cursor.execute("CREATE INDEX IF NOT EXISTS idx_fecha ON santander(fecha_dato);")

# Índice compuesto cliente + fecha
cursor.execute("CREATE INDEX IF NOT EXISTS idx_cliente_fecha ON santander(ncodpers, fecha_dato);")

con.commit()
con.close()

print("✅ Índices creados correctamente.")


✅ Índices creados correctamente.


In [20]:
import pandas as pd
con = sqlite3.connect("Proyecto.db")

# Buscar todos los registros del cliente 123456 en junio 2015
query = """
SELECT *
FROM santander
WHERE ncodpers = 123456 AND fecha_dato = '2015-06-28'
"""
df = pd.read_sql(query, con)
print(df.head())


   fecha_dato  ncodpers ind_empleado pais_residencia sexo  age  fecha_alta  \
0  2015-06-28    123456            N              ES    V   52  1999-02-27   

   ind_nuevo antiguedad  indrel  ... ind_hip_fin_ult1  ind_plan_fin_ult1  \
0        0.0        197     1.0  ...                0                  0   

  ind_pres_fin_ult1 ind_reca_fin_ult1 ind_tjcr_fin_ult1 ind_valo_fin_ult1  \
0                 0                 0                 1                 0   

  ind_viv_fin_ult1 ind_nomina_ult1  ind_nom_pens_ult1  ind_recibo_ult1  
0                0             1.0                1.0                1  

[1 rows x 48 columns]


In [ ]:
query_two = """
SELECT distinct 
FROM santander
WHERE ncodpers = 123456 AND fecha_dato = '2015-06-28'
"""
df = pd.read_sql(query, con)
print(df.head())

## Se crea la tabla clientes con información personal y las columnas de productos las que inician nd_(xyz)_ult1 según indicaciones... (Esto es opcional)

In [17]:
import sqlite3
import pandas as pd

ruta = '/workspaces/proyecto-final/data/train_ver2.csv'
con = sqlite3.connect("Proyecto.db")

chunksize = 100000
for i, chunk in enumerate(pd.read_csv(ruta, chunksize=chunksize, low_memory=False)):
    print(f"Procesando chunk {i+1}")

    # Solo columnas relevantes para la tabla cliente_mes
    cols = ['ncodpers', 'fecha_dato', 'age', 'renta', 'segmento']
    productos = [col for col in chunk.columns if col.startswith('ind_') and col.endswith('_ult1')]

    subset = chunk[cols + productos].copy()
    subset['productos_activos'] = subset[productos].sum(axis=1)

    # Guardar en nueva tabla
    subset.to_sql("cliente_mes", con, if_exists="append", index=False)

print("✅ Tabla cliente_mes creada.")


Procesando chunk 1
Procesando chunk 2
Procesando chunk 3
Procesando chunk 4
Procesando chunk 5
Procesando chunk 6
Procesando chunk 7
Procesando chunk 8
Procesando chunk 9
Procesando chunk 10
Procesando chunk 11
Procesando chunk 12
Procesando chunk 13
Procesando chunk 14
Procesando chunk 15
Procesando chunk 16
Procesando chunk 17
Procesando chunk 18
Procesando chunk 19
Procesando chunk 20
Procesando chunk 21
Procesando chunk 22
Procesando chunk 23
Procesando chunk 24
Procesando chunk 25
Procesando chunk 26
Procesando chunk 27
Procesando chunk 28
Procesando chunk 29
Procesando chunk 30
Procesando chunk 31
Procesando chunk 32
Procesando chunk 33
Procesando chunk 34
Procesando chunk 35
Procesando chunk 36
Procesando chunk 37
Procesando chunk 38
Procesando chunk 39
Procesando chunk 40
Procesando chunk 41
Procesando chunk 42
Procesando chunk 43
Procesando chunk 44
Procesando chunk 45
Procesando chunk 46
Procesando chunk 47
Procesando chunk 48
Procesando chunk 49
Procesando chunk 50
Procesand

In [22]:
# Mostrar primeros registros
df = pd.read_sql("SELECT * FROM cliente_mes LIMIT 5;", con)
print(df)

# Mostrar columnas
col_info = pd.read_sql("PRAGMA table_info(cliente_mes);", con)
print(col_info[['name', 'type']])


   ncodpers  fecha_dato  age      renta            segmento  \
0   1375586  2015-01-28   35   87218.10   02 - PARTICULARES   
1   1050611  2015-01-28   23   35548.74  03 - UNIVERSITARIO   
2   1050612  2015-01-28   23  122179.11  03 - UNIVERSITARIO   
3   1050613  2015-01-28   22  119775.54  03 - UNIVERSITARIO   
4   1050614  2015-01-28   23        NaN  03 - UNIVERSITARIO   

   ind_ahor_fin_ult1  ind_aval_fin_ult1  ind_cco_fin_ult1  ind_cder_fin_ult1  \
0                  0                  0                 1                  0   
1                  0                  0                 1                  0   
2                  0                  0                 1                  0   
3                  0                  0                 0                  0   
4                  0                  0                 1                  0   

   ind_cno_fin_ult1  ...  ind_plan_fin_ult1  ind_pres_fin_ult1  \
0                 0  ...                  0                  0   
1       

In [19]:
import sqlite3
import pandas as pd

# Conexión a la base de datos
con = sqlite3.connect("Proyecto.db")

# Consulta SQL dentro de comillas
query = "SELECT * FROM cliente_mes LIMIT 500000;"

# Ejecutar y obtener resultados en un DataFrame
df = pd.read_sql(query, con)

# Mostrar resultados
print(df)


DatabaseError: Execution failed on sql 'SELECT * FROM cliente_mes LIMIT 500000;': no such table: cliente_mes

In [24]:
cliente_id = 1375586  # cámbialo si no existe



In [25]:
query = f"""
SELECT *
FROM cliente_mes
WHERE ncodpers = {cliente_id}
ORDER BY fecha_dato;
"""

df_cliente = pd.read_sql(query, con)


In [26]:
df_cliente.head(10)


,ncodpers,fecha_dato,age,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,...,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,productos_activos
0,1375586,2015-01-28,35,87218.1,02 - PARTICULARES,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0,1.0
1,1375586,2015-02-28,35,87218.1,02 - PARTICULARES,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0,1.0
2,1375586,2015-03-28,35,87218.1,02 - PARTICULARES,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0,1.0
3,1375586,2015-04-28,35,87218.1,02 - PARTICULARES,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0,1.0
4,1375586,2015-05-28,35,87218.1,02 - PARTICULARES,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,1,2.0
5,1375586,2015-06-28,35,87218.1,02 - PARTICULARES,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,1,2.0
6,1375586,2015-07-28,35,87218.1,02 - PARTICULARES,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,1,2.0
7,1375586,2015-08-28,36,87218.1,02 - PARTICULARES,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0,1.0
8,1375586,2015-09-28,36,87218.1,02 - PARTICULARES,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0,1.0
9,1375586,2015-10-28,36,87218.1,02 - PARTICULARES,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0,1.0
